# Hybrid pipeline (Option 1) — Fast demo

**KAN → VQE → Krylov → FFT → QPE (local simulation)**

This notebook is a compact, runnable demonstration intended for Google Colab / QBraid. It uses a small truncated Hamiltonian (few qubits) and Aer statevector simulator. It demonstrates the concepts with runnable code and plots.

## Setup: Install & imports
Run the cell below to install required packages (takes a minute on Colab).

In [ ]:
!pip install --quiet qiskit qiskit-aer torch scipy numpy matplotlib nbformat

In [ ]:
import os
import math
import numpy as np
import torch
from torch import nn
from scipy.linalg import expm, eig, fractional_matrix_power
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.primitives import Estimator
from qiskit.quantum_info import Operator
from qiskit.extensions import UnitaryGate
from qiskit.algorithms.optimizers import COBYLA

print('Imports successful')

## Build truncated Hamiltonian (small demo)

In [ ]:
N_TRUNC = 6
N_QUBITS = int(np.ceil(np.log2(N_TRUNC)))
DIM = 2 ** N_QUBITS

def ladder_ops(N):
    a = np.zeros((N,N), dtype=complex)
    for n in range(N-1):
        a[n,n+1] = np.sqrt(n+1)
    return a, a.conj().T

a, adag = ladder_ops(N_TRUNC)
x = (a + adag)/np.sqrt(2)
p = 1j*(adag - a)/np.sqrt(2)
H_small = 0.5*(x@p + p@x)
H_small = 0.5*(H_small + H_small.conj().T)

H_pad = np.zeros((DIM,DIM), dtype=complex)
H_pad[:N_TRUNC, :N_TRUNC] = H_small
H_op = Operator(H_pad)

cref_vals, cref_vecs = eig(H_small)
cref_vals = np.real_if_close(cref_vals)
idx = np.argsort(cref_vals)
cref_vals = cref_vals[idx]
print('Classical lowest eigenvalues:', np.round(cref_vals[:6],6))

## KAN surrogate (PyTorch) + active sampling

In [ ]:
class KANSurrogate(nn.Module):
    def __init__(self, in_dim, m_units=48, uni_hidden=8):
        super().__init__()
        self.inner = nn.Linear(in_dim, m_units)
        self.uni_w = nn.Parameter(torch.randn(m_units, uni_hidden) * 0.01)
        self.uni_b = nn.Parameter(torch.zeros(m_units, uni_hidden))
        self.uni_out = nn.Parameter(torch.randn(m_units) * 0.01)
        self.outer = nn.Linear(m_units, 1)
    def forward(self, x):
        z = self.inner(x)
        z_exp = z.unsqueeze(-1)
        hidden = torch.tanh(z_exp * self.uni_w.unsqueeze(0) + self.uni_b.unsqueeze(0))
        hid_mean = hidden.mean(dim=-1)
        uni = hid_mean * self.uni_out.unsqueeze(0)
        out = self.outer(uni)
        return out.squeeze(-1)

# small estimator wrapper using Aer statevector
sim = AerSimulator(method='statevector')
est = Estimator(sim)

n_params = 2 * N_QUBITS

def ansatz_circuit(theta):
    qc = QuantumCircuit(N_QUBITS)
    for i in range(N_QUBITS):
        qc.ry(float(theta[i]), i)
        qc.rz(float(theta[i + N_QUBITS]), i)
    for i in range(N_QUBITS - 1):
        qc.cx(i, i+1)
    return qc

def eval_energy(theta):
    qc = ansatz_circuit(theta)
    res = est.run([qc], [H_op]).result()
    return float(res.values[0].real)

# Active sampling
Theta = []
Evals = []
for i in range(12):
    th = np.random.uniform(-np.pi, np.pi, size=(n_params,))
    e = eval_energy(th)
    Theta.append(th); Evals.append(e)
print('Initial samples done')

kan = KANSurrogate(n_params, m_units=48).to('cpu')

# train & propose few candidates
X = torch.tensor(np.array(Theta), dtype=torch.float32)
y = torch.tensor(np.array(Evals), dtype=torch.float32)
opt = torch.optim.Adam(kan.parameters(), lr=1e-3)
for ep in range(200):
    opt.zero_grad()
    p = kan(X)
    loss = ((p - y)**2).mean()
    loss.backward(); opt.step()

# propose minima by gradient-descent on surrogate
cand = []
for r in range(6):
    th_var = torch.tensor(np.random.uniform(-np.pi, np.pi, size=(1,n_params)), dtype=torch.float32, requires_grad=True)
    optx = torch.optim.Adam([th_var], lr=0.2)
    for s in range(120):
        optx.zero_grad()
        out = kan(th_var)
        out.backward(); optx.step()
    cand.append(th_var.detach().cpu().numpy().reshape(-1))

for c in cand:
    e = eval_energy(c)
    Theta.append(c); Evals.append(e)
print('KAN proposals evaluated')

## Short VQE refinement (COBYLA)

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

best_idx = int(np.argmin(Evals))
init = Theta[best_idx]

def obj(x):
    return eval_energy(x)

cobyla = COBYLA(maxiter=100)
x_opt, val, _ = cobyla.optimize(num_vars=n_params, objective_function=obj, initial_point=init)
print('Refined energy:', val)

## Krylov (classical simulation of overlaps) and FFT spectral scan

In [ ]:
# build Krylov around refined state
qc_opt = ansatz_circuit(x_opt)
sv = sim.run(qc_opt).result().get_statevector(qc_opt)
psi0 = sv[:N_TRUNC]/np.linalg.norm(sv[:N_TRUNC])

kdim = 4
vecs = [psi0]
for k in range(1,kdim):
    v = H_small @ vecs[-1]
    for prev in vecs:
        v = v - np.vdot(prev, v) * prev
    if np.linalg.norm(v) < 1e-12: break
    vecs.append(v/np.linalg.norm(v))

# compute S and Hproj
K = len(vecs)
S = np.zeros((K,K), dtype=complex); Hproj = np.zeros_like(S)
for i in range(K):
    for j in range(K):
        S[i,j] = np.vdot(vecs[i], vecs[j])
        Hproj[i,j] = np.vdot(vecs[i], H_small @ vecs[j])

from scipy.linalg import fractional_matrix_power
S_inv_sqrt = fractional_matrix_power(S, -0.5)
H_eff = S_inv_sqrt @ Hproj @ S_inv_sqrt
vals, vecs_k = eig(H_eff)
vals = np.real_if_close(vals); vals.sort()
print('Krylov approx eigenvals:', np.round(vals,6))

# FFT spectral reconstruction
T_MAX=6.0; N_TIME=256; DELTA_T=T_MAX/N_TIME
U_step = expm(-1j * H_pad * DELTA_T)

# compute C(t)
C_t = np.zeros(N_TIME, dtype=complex)
cur = sv.copy()
for n in range(N_TIME):
    C_t[n] = np.vdot(sv, cur)
    cur = U_step @ cur

window = np.hanning(N_TIME)
C_w = C_t * window
spectrum = np.fft.fftshift(np.fft.fft(C_w))
freqs = np.fft.fftshift(np.fft.fftfreq(N_TIME, d=DELTA_T))
power = np.abs(spectrum)
peaks, _ = find_peaks(power, height=np.max(power)*0.06)
print('FFT peaks (freqs):', np.round(freqs[peaks[:8]],6))

import matplotlib.pyplot as plt
plt.figure(figsize=(8,3)); plt.plot(freqs, power); plt.scatter(freqs[peaks], power[peaks], color='red'); plt.title('FFT spectral power')
plt.show()

## QPE refinement (simulated using exact UnitaryGate)

In [ ]:
from qiskit import QuantumCircuit

U_total = expm(-1j * H_pad * 1.0)
U_gate = UnitaryGate(U_total)

n_count = 5
qc_qpe = QuantumCircuit(n_count + N_QUBITS, n_count)
qc_qpe.initialize(sv.tolist(), list(range(n_count, n_count+N_QUBITS)))
for q in range(n_count): qc_qpe.h(q)
for j in range(n_count):
    powr = 2**(n_count-1-j)
    qc_qpe.append(UnitaryGate(np.linalg.matrix_power(U_total, powr)).control(1), [j] + list(range(n_count, n_count+N_QUBITS)))
# qft dagger
for i in range(n_count//2): qc_qpe.swap(i, n_count-1-i)
for j in range(n_count):
    for m in range(j): qc_qpe.cp(-np.pi/(2**(j-m)), m, j)
    qc_qpe.h(j)
for q in range(n_count): qc_qpe.measure(q,q)

job = sim.run(qc_qpe, shots=1024).result()
counts = job.get_counts(qc_qpe)
most = max(counts.items(), key=lambda kv: kv[1])[0]
phi = int(most,2)/(2**n_count)*2*np.pi
E_est = phi/1.0
print('QPE estimate (simulated):', E_est)

### End of Option 1 demo
This notebook is intentionally compact. You can tweak N_TRUNC, KAN sample counts, and FFT settings to explore behavior.